# 基于用户行为的兴趣数据标签  
  

### 一、数据标签的重要性和定义

##### 1.用户静态标签

1.学校标签  
通过学校的共同性进行推荐，清华的学生更多地推荐清华的学生  

2.专业标签  
通过专业的共同性进行推荐，但是因为有很多交叉学科（比如UI设计就是设计和心理学、计算机的交叉学科，这样可以给用户打上不同的用户标签，推荐多个设计、心理学、计算机的个集或者交集的用户）、相同大类用户（比如文科类用户推荐文科用户、理科类用户推荐理科用户、工科推荐工科或者理科用户） 

3.学院标签
通过学院的共同性进行推荐，经济学院的学生更容易和经济学院的学生形成交流

3.年限推荐  
大学四年时间内，所接触到的各个年级学生，更容易成为密集交流者  

##### 2.用户动态标签
1.内容标签  
基于用户浏览内容的标签，这个时候重点在于“如何从内容提取标签内容”——因为排除音频和视频的计划成本，最直接的是用所浏览和发送的内容的文本，从文本中进行挖掘。
  
*2.情绪标签   
通过用户所浏览的内容的文本进行情感化分析，会根据用户行为，定义这一段时间用户的心理变化，或者说可以掌控整个平台的社交情绪。

ps.*情绪语义分析，需要大量的数据和成本，只可作为参考。

### 二.数据标签抽取算法
##### 一：采用经典的tf-idf算法，虽然一定程度上会有所不足，但是加入权重和词袋(Bag Of Word)层级模型。  
##### 二：运用svm自行进行模型学习，但无法在线动态改变模型，根据词向量和模型的拟合，进行制作。（可用于智能客服的使用）

##### 1.TF-IDF算法  
##### 定义一：出现频率较多的词，会更加与主题相联系（排除掉‘的’、‘是’等停用词）
##### 定义二：越是在其他动态中出现的词语，越是趋向于整个词包层级的下层，越具像化
TF-IDF算法是用于资讯检索与资讯探勘的常用加权技术。tf-idf是一种统计方法，用以评估一个字词对于一个文件或一个语料库中的其中一份文件的重要程度。  

**TF**(Term Frequency):指的是某一个给定的词语在该文件中出现的次数。  
具体表现为词在整体文章次数中出现的频率。

$$ TF_{\omega} = \frac{\omega_{ij}}{\sum\omega_{ij}}$$  

数值越大越能表示这个词更加能够表示整个文章的主题。

**IDF** (Invese Document Frequency):指的是逆向文本频率，表现为所有文本中的出现频率。
具体表现为总文件数目处以包含该词语之文件的数目。
  
$$ IDF = log\frac{N}{N(x)} $$  
  
数值越大，则表示包含这个词条的文档越少，则说明词条具有很好的类别区分能力。  
  
$$ TF-IDF(x) = TF(x) * IDF(x) $$

**例：**  
在某个共有一千个单词的文章中“小球战术”和“詹姆斯”分别出现了8次和12次，则它们的词频分别是0.008和0.012，在整体一千篇文章中“小球战术”出现20次，“詹姆斯”出现100次，“小球战术”的idf系数则为log(1000/20)=1.70，“詹姆斯”的idf系数则为log(1000/100)=1，则说明“小球战术”这个词语更加具有分类的特性，在相关性排名中比“詹姆斯”更高。  

#### 但是有这么一种问题：

##### 动态一：“勇士与骑士在第二场NBA总决赛，居然达到了2-0，詹姆斯实在无力带动骑士啊。。。”
##### 动态二：“勇士不赢，NBA天理不容。”
##### 动态三：“詹姆斯可以说是整个NBA联盟、乃至整个篮球的历史中，都是独一无二无敌的存在。”
##### 动态四：“下面请看耐克篮球和安德玛篮球之间的直接博弈。”
##### 动态五：“我觉得总决赛应该就这样结束了。”

运用分词，以及去掉停用词(stop_word)，得到以下分词:  

**理想状态：**

动态一："骑士","勇士",“NBA”,"总决赛","詹姆斯"  
动态二："勇士","NBA"  
动态三："詹姆斯","NBA",“篮球”  
动态四："篮球"  
动态五："总决赛"  

**实际情况：**

动态一：“骑士”，“勇士”，“NBA”，“总决赛”，“无力”，“带动”，“实在”，“居然”，“达到”  
动态二：“勇士”，“NBA”，“不赢”，“天理不容”  
动态三：“詹姆斯”，“NBA”，“篮球”，“历史”，“独一无二”  
动态四：“耐克”，“篮球”，“安德玛”，“直接”，“博弈”  
动态五：“总决赛”，“应该”，“就这样”，“结束”  

可以看到有很多的干扰词，无法定义主题词，所有的词汇都会进行一次计算，这个时候就要引入 **词袋层级模型**  
(词袋模型：不计算词语先后顺序，在词袋模型中“我喜欢你”和“你喜欢我”是等价的)   

在这里类似于形成一个知识图谱，来确定父子级关系  
如：**篮球->勇士->小球战术->库里**
  
**解决办法**
  
（1）：手动调整层级，如后台兴趣前后级关系，自动进行匹配系数，如“篮球”主题系数2.0，“勇士”主题系数1.4，“小球战术”主题系数1.2，“库里”主题系数0.4，缺点在于，当用户量上升，各自的内容复杂性提升，则不再具有优势（想象以下上万个主题标签）  
  
（2）：**Apriori算法**
Apriori算法是经典的关联规则算法，用以构造词到词的关联图，图中每个节点代表一个词，每条边代表词A到词B的一个关联，说明A和B在概念上是有一定相关性的，其公式是：

$$ 支持度： Support(A\bigcap B) = \frac{Freq(A\bigcap B)}{N} $$  
  
$$ 置信度： Confidence = \frac{Freq(A\bigcap B)}{Freq(A)} $$  
   
支持度是指既有A又有B的概率  
置信度是指A发生的同时B发生的概率
  
对于概念层次较高的词，该词在图中节点的出度很大（和很多词相关联，如“体育”可以关联到“足球”，“篮球”众多体育类型，而在“足球”子集的“英超”之下，就是具体的球员名，而“体育”这个节点是和所有体育运动员的名字是相关联的）  
  
##### 2.svm模型
1.运用tf-idf构建词向量，把提取关键词转换为词向量到原点之间的分类问题。
2.运用目前机器学习分类效果较好的svm模型，来进行分类提取。

#### 示例：

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import jieba
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

模拟两个分类，一个是招聘的详细条件分词，而另一个是整体的代入模型，但是因为tf-idf模型的原因，整体词向量是偏向于骑士这个相对高频词。

In [15]:
_target = []
_data1 = ["招人2名", "5000+", "五险一金", "周末双休", "加班工资"]
_target.append("招聘")

_data2 = ["勇士与骑士在第二场NBA总决赛，居然达到了2-0，詹姆斯实在无力带动骑士啊。。。"]
_target.append("NBA")

_X = []
_Y = []

for _data in _data1:
    _X.append(" ".join(jieba.lcut(_data)))
    _Y.append(0)
    
for _data in _data2:
    _X.append(" ".join(jieba.lcut(_data)))
    _Y.append(1)

print(_X)
    
_v = TfidfVectorizer()
_X = _v.fit_transform(_X)

_clf = SVC(C=1000000.0,gamma='auto',kernel='rbf')
_clf.fit(_X, _Y)

# _data = "骑士与勇士在NBA总决赛大战了两个回合，勇士以2:0领先于骑士"
_data = "骑士"
_x = " ".join(jieba.lcut(_data))
print(_x)
_x = _v.transform([_x])
print(_x)
_y = _clf.predict(_x)
print("-----------------------------------")
print("标签是："+_target[_y[0]])

['招人 2 名', '5000 +', '五险 一金', '周末 双休', '加班 工资', '勇士 与 骑士 在 第二场 NBA 总决赛 ， 居然 达到 了 2 - 0 ， 詹姆斯 实在 无力 带动 骑士 啊 。 。 。']
骑士
  (0, 18)	1.0
-----------------------------------
标签是：NBA
